In [1]:
# Gatherting Initial Data
CVALL_Directory = "/Users/Andrew/Documents/FacialRecognitionRacialBias/Data/RawUnzippedFolders/CVALL"

# Prepping image transformations

from torchvision import transforms
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
    transforms.CenterCrop((224, 224)),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),   #must same as here
     transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Creating Total Dataset
from torchvision import datasets
all_dataset = datasets.ImageFolder(CVALL_Directory, transforms_train)


In [2]:
# Model class
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        
        # Convolutional Layer 1
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        # Batch normalization 1
        self.bn1 = nn.BatchNorm2d(out_channels)

        # Convultional Layer 2
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        # Batch Normalization 2
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Creating the Skip Connection
        # By Default create an identity mapping
        self.shortcut = nn.Sequential()

        # If stride != 1 then the spatial dimensions of the input need to be downsampled 
        # OR
        # If input channels and output channels are not equal, then the shortcut connect must adjust the number of channels
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.conv1(input = x)
        out = self.bn1(input = out)
        out = F.relu(input = out, inplace= True)
        out = self.conv2(input = out)
        out = self.bn2(input = out)
        shortcut = self.shortcut(input = x)
        out += shortcut
        out = F.relu(input = out, inplace= True)
        return out


class ResidualCNN(nn.Module):
    def __init__(self, block_used, number_of_blocks, num_classes):
        super(ResidualCNN, self).__init__()
        self.in_channels = 64

        #IMAGE: 224 x 224 X 3

        # Initial Convolutional Layer
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels= 64, kernel_size= 7, stride= 2, padding = 1)
        
        # Batch Normalization & Max Pooling
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size= 3, stride = 2, padding = 1)

        # Residual Block 1
        self.ResidualBlock1 = self.make_layer(block_used, 64, number_of_blocks[0], stride = 1)
        # Residual Block 2
        self.ResidualBlock2 = self.make_layer(block_used, 128, number_of_blocks[1], stride = 2)
        # Residual Block 3
        self.ResidualBlock3 = self.make_layer(block_used, 256, number_of_blocks[2], stride = 2)
        # Residual Block 4
        # self.ResidualBlock4 = self.make_layer(block_used, 512, number_of_blocks[3], stride = 2)
        
        # Average Pool
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block_used, out_channels, blocks, stride):
        layers = []
        layers.append(block_used(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for block_to_create in range(1, blocks):
            layers.append(block_used(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        # Initial Convolutional Layer
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool(x)

        # Residual Blocks
        x = self.ResidualBlock1(x)
        x = self.ResidualBlock2(x)
        x = self.ResidualBlock3(x)
        # x = self.ResidualBlock4(x)

        # Average Pooling
        x = self.avgpool(x)
        x = torch.flatten(x, 1)  # Flatten the output for the fully connected layer
        x = self.fc(x)

        return x

In [3]:
# Train Helper Function
def train(model, train_loader, criterion, optimizer, device):
    # Set to training mode
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        # move images to GPU 
        images, labels = images.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # forward propogate, compute predicted output based on images
        outputs = model(images)

        # calculate the loss
        loss = criterion(outputs, labels)

        # backwards propogate
        loss.backward()

        # update the model's parameters
        optimizer.step()

        # update running loss
        running_loss += loss.item()

        # calc num of correctly predicted samples
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    # Calculate average training loss and accuracy for epoch
    train_loss = running_loss / len(train_loader)
    train_acc = 100.0 * correct / total

    return train_loss, train_acc

In [4]:
# Validation Helper function
def validate(model, val_loader, criterion, device):
    # Set model to eval mode / tracking variables
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    # disable gradient calc (since we are not training model we are validating / testing)
    with torch.no_grad():
        # iterate over the batches
        for images, labels in val_loader:

            # move images to GPU
            images, labels = images.to(device), labels.to(device)

            # forward propogate
            outputs = model(images)

            #calc loss
            loss = criterion(outputs, labels)

            # calculate running loss
            running_loss += loss.item()

            # calculate correct number of observations
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = running_loss / len(val_loader)
    val_acc = 100.0 * correct / total

    return val_loss, val_acc

In [5]:
# Calculate Weights For Cross Entropy Due To Class Imbalances
# Grab all labels
targets = [sample[1] for sample in all_dataset.samples]

# Calculate class weights
class_counts = Counter(targets)
total_samples = len(targets)
class_weights = {class_idx: total_samples / count for class_idx, count in class_counts.items()}
weights = [class_weights[i] for i in range(len(class_counts))]

# Take calculated weights into a tensor
weights = torch.tensor(weights, dtype=torch.float32).to("cuda")

In [7]:
# Initialize All Dataset

# Stratified K Fold 
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import Subset
import torch.optim as optim
import numpy as np


skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
num_epochs = 50
targets = [sample[1] for sample in all_dataset.samples]


for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(targets)), targets)):
    # Initialize Model Criterion Optimizer
    device = "cuda"
    model = ResidualCNN(ResidualBlock, [1,1,1,1], 4).to(device)
    criterion = nn.CrossEntropyLoss(weight = weights)
    optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

    # grab subset for training based on train indices returned by skf.split()
    train_subset = Subset(all_dataset, train_idx)

    # grab subset for validation based on train indices returned by skf.split()
    val_subset = Subset(all_dataset, val_idx)

    # Now properly Load Data into ingestible form for ResNet
    train_loader = torch.utils.data.DataLoader(train_subset, batch_size=50, shuffle=True, num_workers=0)
    val_loader = torch.utils.data.DataLoader(val_subset, batch_size=50, shuffle=False, num_workers=0)


    for epoch in range(num_epochs):
        # Train model with current train fold
        train_loss, train_acc, = train(model, train_loader, criterion, optimizer, device)

        # Validate Model with current validation fold
        val_loss, val_acc = validate(model, val_loader, criterion, device)

        print(f"Fold: {fold + 1} Epoch: {epoch + 1}")
        print(f"train loss: {train_loss} train_acc: {train_acc}")
        print(f"val loss: {val_loss} val_acc: {val_acc}")



Fold: 1 Epoch: 1
train loss: 1.3512050421457105 train_acc: 37.87540471139891
val loss: 1.317456195089552 val_acc: 39.04889484259879
Fold: 1 Epoch: 2
train loss: 1.301352031715733 train_acc: 40.44881098582115
val loss: 1.2831513166427613 val_acc: 42.01830765795937
Fold: 1 Epoch: 3
train loss: 1.2681714664926769 train_acc: 42.787763760187566
val loss: 1.2472489734490713 val_acc: 44.764456351864254
Fold: 1 Epoch: 4
train loss: 1.2383607205906286 train_acc: 45.31092999888355
val loss: 1.2446180595291985 val_acc: 43.62580933244028
Fold: 1 Epoch: 5
train loss: 1.2094815082868826 train_acc: 47.220051356480965
val loss: 1.2076674898465474 val_acc: 44.630497878990845
Fold: 1 Epoch: 6
train loss: 1.1873892472647027 train_acc: 48.347661047225635
val loss: 1.2165334754519992 val_acc: 46.483590087073004
Fold: 1 Epoch: 7
train loss: 1.1601015111862119 train_acc: 50.27352908339846
val loss: 1.1910732997788323 val_acc: 47.57758428220585
Fold: 1 Epoch: 8
train loss: 1.1381874947826842 train_acc: 52.043

KeyboardInterrupt: 